In [1]:
import json
import requests
import io
import numpy as np
import cv2
import os
from base64 import b64encode, b64decode
from pathlib import WindowsPath
from PIL import Image
import json
import pandas as pd
import time

Utils Fs

In [2]:
server = "http://127.0.0.1:7860" # Check the port number

class CNet2Img():
    def __init__(self,query):
        self.url = server + "/sdapi/v1/txt2img"
        self.body = {
            "prompt": query["prompt"],
            "negative_prompt": "longbody, lowres, bad anatomy, bad hands, missing fingers, extra digit, fewer digits, cropped, worst quality, low quality",
            "seed": query["seed"],
            "subseed": -1,
            "subseed_strength": 0,
            "batch_size": 1,
            "n_iter": 1,
            "steps": query["steps"],
            "cfg_scale": 5,
            "width": query["width"],
            "height": query["height"],
            "restore_faces": False,
            "tiling": False,
            "eta": 0,
            "sampler_index": query["sampler_index"],
            "save_images": False,
            "alwayson_scripts": {
                "controlnet": {
                        "args": [
                            {
                                "input_image": query["img"],
                                "mask": "",
                                "module": query["preprocessor"],
                                "model": query["model"],
                                "weight": 1,
                                "resize_mode": "Scale to Fit (Inner Fit)",
                                "lowvram": False,
                                "processor_res": -1,
                                "threshold_a": -1,
                                "threshold_b": -1,
                                "guidance": 1,
                                "guidance_start": 0,
                                "guidance_end": 1,
                                "guessmode": False
                            }
                        ]
                    },
            }
        }

    def sendRequest(self):
        r = requests.post(self.url, json=self.body)
        return r.json()

def readImage(path):
    img = cv2.imread(path)
    retval, buffer = cv2.imencode('.jpg', img)
    b64img = b64encode(buffer).decode("utf-8")
    return b64img

def createOutputFolder(name):
    success = False
    y = 0
    while success == False:
        try:
            out_dir = os.path.join("outputs", f"{name}_{'{:0>4}'.format(y)}")
            os.mkdir(out_dir)
            success = True
        except:
            y += 1
    return out_dir

def loopImg (name, b64img, steps) :

    # Create folder for outputs
    out_dir = createOutputFolder(name)
    prompts = [] 
        
    # Save first input
    image = Image.open(io.BytesIO(b64decode(b64img)))
    filepath = out_dir+'/iter_0000.png'
    image.save(filepath)

    # Set query
    query = {
            "prompt": "an architectural drawing, josan gonzales and dan mumford, engulfed in technology, 2019, risograph, cinemascope panorama, by William Yellowlees, 2 0 1 0 s, hydropunk, panfuturism, detailed scan, chinatownc",
            "steps": 30,
            "sampler_index": "DPM++ 2M Karras", #Euler a
            "width": 512,
            "height": 512,
            "seed": -1,
            "img": b64img
        }

    # Start loop
    for j in range(1, steps):
        
        # Launch SD
        js = depth2Img(query).sendRequest()
        b64img = js['images'][0]

        # Save image
        image = Image.open(io.BytesIO(b64decode(b64img)))
        filepath = out_dir+'/iter_'+ '{:0>4}'.format(j) +'.png'
        image.save(filepath)
    

Loop

In [ ]:
path = "" # path to the input image
name = "" # name of the generation
n_iter = 5 # number of iterations

b64img = readImage(path)

loopImg (name, b64img, 20) 
for i in range(1, n_iter):
    loopImg (name, b64img, 10) 